In [1]:
# 读取数据集，分析2018年的销售数据
import pandas as pd
import chardet
import xgboost
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn import metrics




def encode_text_dummy(df, name, delete =  False):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    if delete:    
        df.drop(name, axis=1, inplace=True)


In [2]:
df = pd.read_csv("/home/gang/CSV/2018.csv",encoding='gb18030',low_memory=False)

In [9]:
df.shape

(782957, 13)

In [11]:
df.shape[0]-df.count() #缺失值的数量

店铺编号    1866
店铺简称       0
店铺省市       0
店铺地址    8468
货号      1866
货品名称    1866
销售日期    1866
颜色编号    1866
颜色说明    1866
尺码      1866
吊牌价     3732
销售价格    3734
数量      3732
dtype: int64

In [12]:
df = df.dropna()

In [13]:
df['month'] = pd.to_datetime(df['销售日期']).apply(lambda x:x.strftime("%m"))
df["month"] = df["month"].astype("int")

encode_text_dummy(df, name = "货品名称" )
encode_text_dummy(df, name = "month" )

In [14]:
encode_text_dummy(df, name = "颜色编号" )
encode_text_dummy(df, name = "尺码")

In [20]:
import re
df["省或市"] = df["店铺省市"].apply(lambda x : re.split("省|市|区",x)[0])

In [22]:
df["discount"] = df["销售价格"]/df["吊牌价"]

In [23]:
drop_columns = df.columns[:10]

In [24]:
drop_columns

Index(['店铺编号', '店铺简称', '店铺省市', '店铺地址', '货号', '货品名称', '销售日期', '颜色编号', '颜色说明',
       '尺码'],
      dtype='object')

In [21]:
df["省或市"].value_counts()

上海         299395
江苏         120625
广东          58836
天津          37449
辽宁          34931
贵州          27725
河南          20560
湖北          15775
河北          15015
福建          14067
北京          13485
内蒙古自治       11763
山东          11537
安徽          11174
浙江          11078
黑龙江         10482
广西壮族自治       8749
云南           8518
陕西           7513
新疆维吾尔自治      6686
重庆           4947
青海           3900
吉林           3529
山西           3393
海南           2274
湖南           1784
南京           1560
四川           1553
甘肃           1307
宁夏回族自治       1145
Name: 省或市, dtype: int64

In [31]:
df_coat = df[["货品名称-长款羽绒服", "month-1", "month-2", "month-3", "month-4", "month-5", "month-6", "month-7", "month-8", "month-9", "month-10", "month-11"]]

In [32]:
df_coat.corr()

,货品名称-长款羽绒服,month-1,month-2,month-3,month-4,month-5,month-6,month-7,month-8,month-9,month-10,month-11
货品名称-长款羽绒服,1.000000,0.081800,0.027166,-0.011463,-0.016448,-0.013702,-0.012331,-0.015237,-0.015246,-0.013853,-0.008766,-0.001264
month-1,0.081800,1.000000,-0.107739,-0.109183,-0.115711,-0.118660,-0.114776,-0.105785,-0.101681,-0.096966,-0.106301,-0.012840
month-2,0.027166,-0.107739,1.000000,-0.111082,-0.117724,-0.120723,-0.116772,-0.107625,-0.103450,-0.098652,-0.108150,-0.013063
month-3,-0.011463,-0.109183,-0.111082,1.000000,-0.119301,-0.122341,-0.118337,-0.109067,-0.104836,-0.099974,-0.109599,-0.013238
month-4,-0.016448,-0.115711,-0.117724,-0.119301,1.000000,-0.129656,-0.125412,-0.115589,-0.111105,-0.105952,-0.116153,-0.014029
month-5,-0.013702,-0.118660,-0.120723,-0.122341,-0.129656,1.000000,-0.128608,-0.118534,-0.113936,-0.108652,-0.119112,-0.014387
month-6,-0.012331,-0.114776,-0.116772,-0.118337,-0.125412,-0.128608,1.000000,-0.114654,-0.110207,-0.105096,-0.115214,-0.013916
month-7,-0.015237,-0.105785,-0.107625,-0.109067,-0.115589,-0.118534,-0.114654,1.000000,-0.101574,-0.096863,-0.106189,-0.012826
month-8,-0.015246,-0.101681,-0.103450,-0.104836,-0.111105,-0.113936,-0.110207,-0.101574,1.000000,-0.093106,-0.102069,-0.012328
month-9,-0.013853,-0.096966,-0.098652,-0.099974,-0.105952,-0.108652,-0.105096,-0.096863,-0.093106,1.000000,-0.097336,-0.011757


In [26]:
pd.get_dummies(df["month"]).columns

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], dtype='int64')